# Proyecto Final SQL 

El objetivo de este proyecto es analizar la base de daros de una aplicación de lectura de libros con el fin de obtener información clave sobre, libros, autores, editoriales y el comportamiento de usuarios para generar una propuesta de valor que fortalezca el diseño de un nuevo producto. 

## Conexión con la base de datos

In [85]:
import pandas as pd
from sqlalchemy import create_engine
db_config = {'user': 'practicum_student',
             'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7',
             'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com',
             'port': 5432, 
             'db': 'data-analyst-final-project-db'
            }
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
engine = create_engine(connection_string, connect_args={'sslmode':'require'})



1. Se siguieron las instrucciones para crear la conexión entre Python y SQL. 

## Exploración de tablas 

In [86]:

query = "SELECT * FROM books LIMIT 5"
pd.io.sql.read_sql(query, con = engine)




,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [87]:
query = "SELECT * FROM authors LIMIT 5"
pd.io.sql.read_sql(query, con = engine)


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [88]:
query = "SELECT * FROM publishers LIMIT 5"
pd.io.sql.read_sql(query, con = engine)


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [89]:
query = "SELECT * FROM ratings LIMIT 5"
pd.io.sql.read_sql(query, con = engine)


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [90]:
query = "SELECT * FROM reviews LIMIT 5"
pd.io.sql.read_sql(query, con = engine)


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


1. Se hizo una exploración de cada tabla.  

## Número de libros publicados después del 1 de enero de 2000

In [91]:

query = """
SELECT COUNT(*) AS books_published_after_2000
FROM books
WHERE publication_date::date > DATE '2000-01-01';
"""


In [92]:
df_books_after_2000 = pd.io.sql.read_sql(query, con=engine)
print("El número de libros publicados después del 1 de enero de 2000 fueron:")
df_books_after_2000


El número de libros publicados después del 1 de enero de 2000 fueron:


,books_published_after_2000
0,819


1. Se contaron las filas, se nombró a la columna con el resultado de la consulta, se estableció la tabla donde se buscaron los datos, se agregó la condicional WHERE para contar los libros que su fecha de publicación fuera después del 1 de enero del 2000.
2. Se ejecutó la consulta con la conexión.
3. Se mostró el resultado.  

Conclusión: 
El resultado arrojó 819 libros publicados después del año 2000. Lo cual, sugiere la plataforma cuenta con una buena cantidad de libros actualizados. 


## Número de reseñas por usuario y calificación promedio para cada libro 

In [93]:


query = """
SELECT b.book_id,
       b.title,
       COUNT(DISTINCT r.review_id) AS reviews_count,
       AVG(rt.rating) AS avg_rating
FROM books b
LEFT JOIN reviews r ON b.book_id = r.book_id
LEFT JOIN ratings rt ON b.book_id = rt.book_id
GROUP BY b.book_id, b.title
ORDER BY reviews_count DESC, b.title ASC
LIMIT 10;
"""



In [94]:

df_reviews_and_ratings = pd.read_sql(query, con=engine)
df_reviews_and_ratings.head(10)


,book_id,title,reviews_count,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,207,Eat Pray Love,6,3.395833
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,497,Outlander (Outlander #1),6,4.125000
5,627,The Alchemist,6,3.789474
6,656,The Book Thief,6,4.264151
7,673,The Catcher in the Rye,6,3.825581
8,695,The Curious Incident of the Dog in the Night-Time,6,4.081081
9,696,The Da Vinci Code (Robert Langdon #2),6,3.830508


1. Se mostraron las columnas de book_id y title. Con COUNT Y DISTINTC se contaron las reseñas de texto evitando duplicados. Finalmente, con AVG(rt.rating) se calculó el promedio de calificaciones.
2. De la tabla de libros se conectó con las reseñas (reviews) y las calificaciones (ratings).
3. Se agruparon y se ordenaron por los que tienen más reseñas.
4. Se ejecutó la conexión y se mostró.  

Conclusión: La consulta arrojó el resultado correcto, es decir, devuelve de los libros más reseñados a los menos con el promedio de cada libro. Este orden en la búsqueda muestra resultados interesantes, primero que los libros con mayores reseñas en la base de datos tienen de 6 a 7 reseñas. Asimismo, se puede observar que los promedios varian, entre quienes tienen buen promedio de calificación de un 4 y entre quienes tienen opiniones divididas de un 3. 

## Editorial con libros de más de 50 páginas 

In [95]:

query = """
SELECT publishers.publisher,
       COUNT(books.book_id) AS books_more_50
FROM books
JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher
ORDER BY books_more_50 DESC
LIMIT 1;
"""



In [96]:
df_publishers_top = pd.read_sql(query, con=engine)
print("La editorial con el mayor número de libros publicados con más de 50 páginas es:")
df_publishers_top

La editorial con el mayor número de libros publicados con más de 50 páginas es:


,publisher,books_more_50
0,Penguin Books,42


1. Con el nombre de la editorial, contamos con COUNT los libros publicados, y se le dió nombre a la columna nueva.
2. Se utilizó JOIN usando la columna publisher_id para saber la editorial de cada libro. Con WHERE se filtraron los libros con más de 50 páginas. Con GROUP  BY se la editorial . Finalmente se ordenó en forma descendente y que limitó a devolver una sola editorial.
3. Se hizo la conexión y se mostró

Conclusiones: La consulta arrojó que Penguin Books lidera el catalogo en libros largos, es decir, con más de 50 páginas. 

## Autor con mayor calificación promedio (solo libros con más de 50 calificaciones)

In [97]:

query = """
SELECT auth.author,
       AVG(rating.rating) AS avg_author_rating
FROM authors auth
JOIN books   book ON book.author_id = auth.author_id
JOIN ratings rating ON rating.book_id   = book.book_id
WHERE book.book_id IN (
    SELECT ratingbook.book_id
    FROM ratings ratingbook
    GROUP BY ratingbook.book_id
    HAVING COUNT(ratingbook.rating_id) >= 50
)
GROUP BY auth.author
ORDER BY avg_author_rating DESC, auth.author ASC
LIMIT 1;
"""


df_author_highest = pd.read_sql(query, con=engine)
print("Autor con mejor promedio considerando libros con ≥ 50 calificaciones:")
df_author_highest


Autor con mejor promedio considerando libros con ≥ 50 calificaciones:


,author,avg_author_rating
0,J.K. Rowling/Mary GrandPré,4.287097



1. Se filtraron los libros que tenían al menos 50 calificaciones. Con esos libros se calculó el promedio de calificaciones de cada autor, se agrupo por autor de forma descendente. 
2. Se hace la conexión y se muestra el resultado. 


Conclusión: La consulta arrojó que J.K. Rowling/Mary GrandPré (posiblemente, autora e ilustadora) es la autora con mejor promedio de calificaciones, con 4.2, considerando libros que suman al menos 50 calificaciones. 

## Número promedio de reseñas entre usuarios que calificaron más de 50 libros 

In [98]:

query = """
WITH intensive_users AS (
    SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT(rating_id) >= 50
),
reviews_intensive_users AS (
    SELECT iusers.username,
           COUNT(rev.review_id) AS review_count
    FROM intensive_users iusers
    LEFT JOIN reviews rev
      ON rev.username = iusers.username
    GROUP BY iusers.username
)
SELECT AVG(review_count)::numeric(10,2) AS average_number_reviews
FROM reviews_intensive_users;
"""



In [99]:
df_average_reviews = pd.read_sql(query, con=engine)
print("El promedio de reseñas entre usuarios que calificaron más de 50 libros es:")
df_average_reviews

El promedio de reseñas entre usuarios que calificaron más de 50 libros es:


,average_number_reviews
0,24.22


1. Se filtraron a los usuarios que calificaron más de 50 libros. Después se contaron el número de reseñas que escribió cada uno. Con esos datos se calculó el promedio de reseñas, a esto se nombró average_number_reviews.
2. Se hizo la conexión y se mostró. 

Conclusiones: La consulta devolvió que los usuarios han calificado más de 50 libros escribieron en promedio 24 reseñas de texto. Lo cual, sugiere que una importante cantidad de usuarios no solo califican los libros sino también dejan reseñas sobre estos. 

## Conclusión general

En suma, el análisis muestra que la base de datos tiene una oferta amplia y reciente, con 819 libros publicados después del año 2000. Además, Penguin Books lidera como editorial más activa y J.K. Rowling/Mary GrandPré (posiblemente, autora e ilustadora) se destaca como la autora mejor valorada, con un promedio de 4.2 estrellas. En cuanto a los usuarios, los más activos no solo califican más de 50 libros, sino que también escriben en promedio 24 reseñas de texto. En conjunto, estos hallazgos proporcionan información valiosa para diseñar estrategias de recomendación, promoción editorial y de comunidad de usuarios en un nuevo producto digital de lectura.